In [1]:
import numpy as np
import h5py
N_set=1000
filepath='./wlan_train_set_joint_tmp.mat'
f=h5py.File(filepath,'r')
train_data=np.array(f['Train_data'][:])
time_label=np.array(f['Train_timing_label_onehot'][:])
cfo_label=np.array(f['Train_cfo_label'][:])/1e3



In [2]:
from keras.models import Model
from keras import models
from keras import layers
from keras import Input
from keras import optimizers
from keras.layers import Dropout
from keras.layers import BatchNormalization
import pydot
import graphviz
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

input_data=Input(shape=(11,800,1))
x=layers.Conv2D(16,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(input_data)
x=layers.MaxPooling2D((1,4),padding='same')(x)
x=layers.Conv2D(32,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(x)
x=layers.MaxPooling2D((1,4),padding='same')(x)

cfo_pred=layers.Conv2D(64,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(x)
cfo_pred=layers.MaxPooling2D((1,4),padding='same')(cfo_pred)
cfo_pred=layers.Conv2D(128,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(cfo_pred)

cfo_pred_skip1=layers.Conv2D(128,(11,1),strides=(1,4),padding='same',activation='relu',kernel_initializer='he_normal')(x)
cfo_pred=layers.add([cfo_pred_skip1,cfo_pred])
cfo_pred=layers.MaxPooling2D((1,4),padding='same')(cfo_pred)

cfo_pred_skip2=layers.Conv2D(512,(11,1),strides=(1,4),padding='same',activation='relu',kernel_initializer='he_normal')(cfo_pred)
cfo_pred=layers.Conv2D(256,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(cfo_pred)
cfo_pred=layers.MaxPooling2D((1,4),padding='same')(cfo_pred)
cfo_pred=layers.Conv2D(512,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(cfo_pred)
cfo_pred=layers.add([cfo_pred_skip2,cfo_pred])

cfo_pred=layers.Flatten()(cfo_pred)
cfo_pred=layers.Dense(512,activation='relu',kernel_initializer='he_normal')(cfo_pred)
cfo_pred=layers.Dense(1,name='cfo')(cfo_pred)

time_pred=layers.Conv2D(64,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(x)
time_pred=layers.MaxPooling2D((2,2),padding='same')(time_pred)
time_pred=layers.Conv2D(128,(3,3),padding='same',activation='relu',kernel_initializer='he_normal')(time_pred)
time_pred=layers.MaxPooling2D((2,2),padding='same')(time_pred)
time_pred=layers.Flatten()(time_pred)
time_pred=layers.Dense(160,activation='softmax',name='time')(time_pred)

model= Model(input_data,[cfo_pred,time_pred])

model.summary()
##plot_model(model,to_file='network.png',show_shapes=True,show_layer_names=False)

Using TensorFlow backend.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 11, 800, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 11, 800, 16)  160         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 11, 200, 16)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 11, 200, 32)  4640        max_pooling2d_1[0][0]            
___________________________________________________________________________________________

In [3]:
model.compile(optimizer=optimizers.Adam(lr=0.001),
                loss={'cfo':'mse','time':'categorical_crossentropy'},
                loss_weights={'cfo':1,'time':10000000},
                metrics={'cfo':'mse','time':'accuracy'})
model.fit(train_data,{'cfo':cfo_label,'time':time_label},epochs=300,batch_size=200)


Epoch 1/300
1000/1000 [==============================] - 2s 2ms/step - loss: 50911796.0000 - cfo_loss: 10337.7754 - time_loss: 5.0901 - cfo_mse: 10337.7754 - time_accuracy: 0.0160
Epoch 2/300
1000/1000 [==============================] - 0s 309us/step - loss: 47342149.6000 - cfo_loss: 12644.1562 - time_loss: 4.7330 - cfo_mse: 12644.1572 - time_accuracy: 0.0650
Epoch 3/300
1000/1000 [==============================] - 0s 313us/step - loss: 42707746.4000 - cfo_loss: 9011.6660 - time_loss: 4.2699 - cfo_mse: 9011.6660 - time_accuracy: 0.0960
Epoch 4/300
1000/1000 [==============================] - 0s 322us/step - loss: 35914028.8000 - cfo_loss: 5855.0908 - time_loss: 3.5908 - cfo_mse: 5855.0908 - time_accuracy: 0.2150
Epoch 5/300
1000/1000 [==============================] - 0s 320us/step - loss: 28517560.0000 - cfo_loss: 3719.9656 - time_loss: 2.8514 - cfo_mse: 3719.9656 - time_accuracy: 0.3030
Epoch 6/300
1000/1000 [==============================] - 0s 304us/step - loss: 22717586.8000 - cf

1000/1000 [==============================] - 0s 319us/step - loss: 584948.5250 - cfo_loss: 16.3089 - time_loss: 0.0585 - cfo_mse: 16.3089 - time_accuracy: 0.9900
Epoch 92/300
1000/1000 [==============================] - 0s 319us/step - loss: 504359.0125 - cfo_loss: 19.0998 - time_loss: 0.0504 - cfo_mse: 19.0998 - time_accuracy: 0.9890
Epoch 93/300
1000/1000 [==============================] - 0s 318us/step - loss: 475279.0094 - cfo_loss: 14.6208 - time_loss: 0.0475 - cfo_mse: 14.6208 - time_accuracy: 0.9890
Epoch 94/300
1000/1000 [==============================] - 0s 318us/step - loss: 355229.9344 - cfo_loss: 14.7758 - time_loss: 0.0355 - cfo_mse: 14.7758 - time_accuracy: 0.9920
Epoch 95/300
1000/1000 [==============================] - 0s 317us/step - loss: 234667.4562 - cfo_loss: 13.4619 - time_loss: 0.0235 - cfo_mse: 13.4619 - time_accuracy: 0.9950
Epoch 96/300
1000/1000 [==============================] - 0s 326us/step - loss: 434368.0094 - cfo_loss: 13.0887 - time_loss: 0.0434 - cfo_

1000/1000 [==============================] - 0s 316us/step - loss: 3643.0333 - cfo_loss: 8.8922 - time_loss: 3.6341e-04 - cfo_mse: 8.8922 - time_accuracy: 1.0000
Epoch 183/300
1000/1000 [==============================] - 0s 319us/step - loss: 3645.6394 - cfo_loss: 9.2141 - time_loss: 3.6364e-04 - cfo_mse: 9.2141 - time_accuracy: 1.0000
Epoch 184/300
1000/1000 [==============================] - 0s 321us/step - loss: 3535.5191 - cfo_loss: 7.0630 - time_loss: 3.5285e-04 - cfo_mse: 7.0630 - time_accuracy: 1.0000
Epoch 185/300
1000/1000 [==============================] - 0s 327us/step - loss: 3519.7970 - cfo_loss: 7.8389 - time_loss: 3.5120e-04 - cfo_mse: 7.8389 - time_accuracy: 1.0000
Epoch 186/300
1000/1000 [==============================] - 0s 317us/step - loss: 3484.3293 - cfo_loss: 6.7384 - time_loss: 3.4776e-04 - cfo_mse: 6.7384 - time_accuracy: 1.0000
Epoch 187/300
1000/1000 [==============================] - 0s 318us/step - loss: 3432.8161 - cfo_loss: 5.8896 - time_loss: 3.4269e-04 

1000/1000 [==============================] - 0s 301us/step - loss: 1417.7499 - cfo_loss: 11.6370 - time_loss: 1.4061e-04 - cfo_mse: 11.6370 - time_accuracy: 1.0000
Epoch 275/300
1000/1000 [==============================] - 0s 306us/step - loss: 1398.3610 - cfo_loss: 16.8823 - time_loss: 1.3815e-04 - cfo_mse: 16.8823 - time_accuracy: 1.0000
Epoch 276/300
1000/1000 [==============================] - 0s 299us/step - loss: 1381.6891 - cfo_loss: 12.0001 - time_loss: 1.3697e-04 - cfo_mse: 12.0001 - time_accuracy: 1.0000
Epoch 277/300
1000/1000 [==============================] - 0s 299us/step - loss: 1378.2570 - cfo_loss: 12.9619 - time_loss: 1.3653e-04 - cfo_mse: 12.9619 - time_accuracy: 1.0000
Epoch 278/300
1000/1000 [==============================] - 0s 305us/step - loss: 1360.8933 - cfo_loss: 12.1733 - time_loss: 1.3487e-04 - cfo_mse: 12.1733 - time_accuracy: 1.0000
Epoch 279/300
1000/1000 [==============================] - 0s 307us/step - loss: 1360.2555 - cfo_loss: 16.0516 - time_loss: 

In [2]:
import numpy as np
import h5py
SNR=range(-15,21,3)
for SNR_loop in SNR:
    filepath='./wlan_test_set_joint_SNR(%d)_tmp.mat' % SNR_loop
    f=h5py.File(filepath,'r')
    test_data=np.array(f['Test_data'][:])
    test_time_label=np.array(f['Test_timing_label_onehot'][:])
    test_cfo_label=np.array(f['Test_cfo_label'][:])/1e3
    a=model.evaluate(test_data, [test_cfo_label,test_time_label])
    print(a)

NameError: name 'model' is not defined

In [9]:
test_time_label.shape

(100, 160)